#Hack Foundation Problem:

ML model for tagging transactions.


#Setup

In [147]:
%%capture

#!pip install torch.     # commented because it is pre-installed in Colab
!pip install torchtext
!pip install transformers
#!pip install numpy      # commented because it is pre-installed in Colab
!pip install portalocker
!pip install pandas
!pip install numpy


In [149]:
import copy
import random

import torch
import numpy as np
import pandas as pd

from torchtext import data as torchtext_data
from torch import nn

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, BertConfig, BertModel, BertTokenizer
from sklearn.model_selection import train_test_split


In [150]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device("mps")               # in case you run on a local Mac with metal performance shaders (setup/support is up to you)
device

device(type='cuda')

In [151]:
torch.manual_seed(10)
np.random.seed(10)

In [152]:
#mount Google Drive
import os
from google.colab import drive
drive.mount('/content/drive')

# Define your directory
output_folder = "/content/drive/MyDrive/Hack for Impact 2024/"

# Ensure the output directory exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Data

In [153]:
import csv
import json

def get_additional_memo(csv_line: list[str]) -> str:
  """
  Checks the check, donation, invoice, and transfer memo for any additional memo

  Returns an empty string if all of those are also empty
  """
  if csv_line[6] != '':
    return csv_line[6]

  if csv_line[7] != '':
    return csv_line[7]

  if csv_line[8] != '':
    return csv_line[8]

  if csv_line[9] != '':
    return csv_line[9]

def get_dataset():
  """
  Reads the CSV file and parses all of them into an array of dictionaries
  """

  data = []
  with open('/content/drive/MyDrive/Hack for Impact 2024/transactions3.csv', mode ='r')as file:
    csvFile = csv.reader(file)
    first = True
    for line in csvFile:
      if first:
        first = False
        continue
      try:
        data.append({
          "amount_cents": line[0],
          "memo": line[1],
          "date": line[2],
          "type": line[3],
          "tags": json.loads(line[4].replace("'", '"').replace('✈️\\xa0', '🛫')),
          "org_category": line[4],
          "org_id": line[5],
          "additional_memo": get_additional_memo(line)
        })
      except:
        print(line)

  return data

In [154]:
data = get_dataset()

In [164]:
small_data = [obs for obs in data if obs['tags'] != []]

In [159]:
#optional label map, unused for now.
label_map = {
    "💰 Funding": "Funding",
    "2024BakeSale": "Events",
    "🎁 Travel Stipends": "Travel",
    "🍕 Food": "Food",
    "🎾Outernet Guilds": "Teambuilding",
    "Prizes": "Events",
    "💻 Network Equipment Purchase": "IT",
    "♾️ Subscription/bill": "Subscription",
    "Marketing mail campaign": "Marketing",
    "✈️ Team Travel": "Travel",
    "🥗 Team Food": "Food",
    "Sponsorships": "Funding",
    "ComicConBakeMay2023": "Events",
    "💸 Donations": "Funding",
    "Business": "Misc",
    "↪️ Reimbursement": "Reimbursement",
    "💳 Misc. Expenses": "Misc",
    "sponsorship": "Funding",
    "🌯 Team Food": "Food",
    "👕 Swag": "Marketing",
    "🧩 development of things doing stuff": "Operations",
    "2SM 2023": "Events",
    "Tax": "Taxes",
    "2SM2023": "Misc",
    "🔌 Supplies": "IT",
    "Mechanical": "IT",
    "💸 Sponsor": "Funding",
    "Funding": "Funding",
    "🌱 Sprig": "Misc",
    "🧘 Healthcare": "Labor",
    "Programing": "Labor",
    "⚙️ Supplies": "IT",
    "🎁 Travel Stipend": "Travel",
    "Marketing/Logo": "Marketing",
    "HCBfees": "Subscription",
    "Oxfam": "Misc",
    "📣 Marketing": "Marketing",
    "Corporate Rental Fee Income": "Operations",
    "Test Wiz": "Misc",
    "🛫Travel Stipend": "Travel",
    "🛫 Flights": "Travel",
    "🔌 Logistics Supplies": "IT",
    "🎨Blot": "Operations",
    "Reimbursement": "Reimbursement",
    "📝 Supplies": "Operations",
    "Travel/Meals": "Travel",
    "Sponsor": "Funding",
    "🌉 Assemble": "Misc",
    "Marketing 📈": "Marketing",
    "Hack Club/HCB HQ": "Subscription",
    "🧰 Supplies": "Operations",
    "🏠 Office": "IT",
    "Equipment": "IT",
    "✨ Workshops & Activities Supplies": "Operations",
    "Supplies": "Operations",
    "💰 Team Expenses Repaid": "Reimbursement",
    "🤡 Disputed": "Refund",
    "📋 Advertisement": "Marketing",
    "⏪ Refunded Charge": "Refund",
    "🍱 Food": "Food",
    "✈️ Travel": "Travel",
    "💸 Payroll": "Labor",
    "🏆Prizes": "Events",
    "HCB Fee": 'Subscription',
    'event meal': 'Food',
    '🧃 Drinks': 'Food',
    '💰Contractor Payments': 'Labor',
    'HQ': 'Subscription',
    '💻 Employee hardware':'IT',
    '💸 De-funding':'Funding',
    'grants': 'Funding',
    'Vendor Payment': 'Labor',
    '2SM2024': 'Misc',
    'Feb2023Bake': 'Sales',
    'Email': 'IT',
    '💰 Grants': 'Funding',
    '🥤Snacks & Drinks': 'Food',
    'Merch': 'Marketing',
    'Fundraising': 'Funding',
    '🏺 Decorations': 'Marketing',
    'Event': 'Events',
    'Grant': 'Funding',
    'Focus Facilitator': 'Misc',
    '📈Marketing': 'Marketing',
    'Promo': 'Marekting',
    'Toy Drive': 'Events',
    'Prize money':'Events',
    '🍝Cookhouse': 'Food',
    'Burning Man Participant':'Events',
    '4 Hour Round Trip UC Davis Veterinary Clinic Road Trip Volunteer Meal':'Food',
    '🎊 Prizes':'Events',
    '🏆 Prizes':'Events',
    '❤️ Team Mementos':'Marketing',
    'Food':'Food',
    '💝 Donations':'Funding',
    'Test':'Misc',
    'Donation':'Funding',
    '💸 Sponsorship':"Funding",
    'Hack Clubber donations': 'Funding',
    '💻 Infra':'IT',
    'WDN': 'Misc',
    '🌈 Fun Supplies': 'Teambuilding',
    '🚀 Meeting Power-Ups': 'Teambuilding',
    '💼 Contractors': 'Labor',
    'Homepod':'IT',
    '💳 Reimbursement':'Reimbursement',
    '📦 Shipping': 'Operations',
    '🏦 HQ': 'Subscription',
    'Indigenous Food Lab': 'Food',
    'Marketing': 'Marketing',
    'test':'Misc',
    '🧪 COVID-19 Supplies': 'Operations',
    'Hardware': 'Operations',
    'Hardware, misc 🔩': 'Operations',
    '💰 Donations': 'Funding',
    'Venue': 'Events',
    'Prize':'Events',
    'Prizes 🏆':'Events',
    'Insurance':'Operations',
    '💰 Sponsorship / Grant':'Funding',
    '🌟Swag':'Marketing',
    'TRANSFER':'Operations',
    '📜 Scrolls':'Misc',
    '🏢 Employee Perks':'Labor',
    '🌤️ Insurance':'Operartions',
    '🚀 W':'Misc',
    '🍇 Catering':'Food',
    'Sponsors':'Funding',
    'Servers':'Events',
    'Printing':'Marketing',
    '👕Swag':'Marketing',
    '🏗️ Infrastructure':'Operations',
    '🎈 Merch':'Marketing',
    'Funds':'Funding',
    '💰 Sponsorship':'Funding',
    '✨ Swag':'Marketing',
    'Volunteer Food':'Food',
    'Books':'Misc',
    'Dog Food':'Food',
    'should be reimbursed by Iberia because flight was cancelled':'Travel',
    'NSP Volunteer':'Labor',
    'Networking Donor Outreach Event':'Events',
    'Individual donation':'Funding',
    '🎤 Episodes':'Marketing',
    'shirts/ swag':'Marketing',
    '💂 Security':'Operations',
    'Maps':'Misc',
    'Merchandise/Spirit Wear':'Marketing',
    'Fee':'Misc',
    '📦 Supplies':'Operations',
    'Donations 💵💵':'Funding',
    'haunted house':'Events',
    '🍕Food':'Food',
    'eBay':'Operations',
    '🌲 Social Media & Advertising':'Marketing',
    'End of WHW :(':'Misc',
    'Two Spirit Wellness Week':'Events',
    '🔧 Equipment':'Operations',
    '🌐 Domains':'IT',
    '✈️Travel Expense':'Travel',
    '🌐Website':'IT',
    '🍜 Catering':'Food',
    '👥 Team Expenses':'Reimbursement',
    'Book Repair Supplies':'Misc',
    'YCJF':'Misc',
    'Zen Master':'Misc',
    '🎮Games':'Teambuilding',
    'Kit and Kickoff/Registration':'Events',
    'Event Costs':'Events',
    '🍉Food & Snacks':'Food',
    '🎉 Fun':'Teambuilding',
    'Orange Pi':'Food',
    '🥪 Food':'Food',
    'Essentials':'Operations',
    '🛠️ Tools for Team':'IT',
    'Viasat':'Misc',
    'Postage':'Operations',
    '🏛️ Staff Resources':'Operations',
    'CliDef':'Misc',
    '🎁 Prizes':'Events',
    '🏆Prize':'Events',
    'team snacks':'Food',
    'Flat USB':'IT',
    'Challenge Coin':'Misc',
    'Individual Donation':'Funding',
    'Personnel Expense':'Labor',
    'Fundraiser Revenue':'Funding',
    'community calls':'Marketing',
    '😄 Fun!':'Teaambuilding',
    '🏆 Prize':'Events',
    'Maria flight Kenya - to be reimbursed by FFNPT':'Travel',
    '🎉 Prizes':'Events',
    'Domain':'IT',
    '🎈 Decor':'Events',
    'TharpHoodies':'Marketing',
    '💻 Maintenance':'IT',
    'Refunded':'Reimbursement',
    'registration':'Events',
    'FTeetersHoodie':'Marketing',
    '🚕 Transport':'Travel',
    'remimbursement':'Reimbursement',
    '💻Laptop':'IT',
    'Electric Feather':'Misc',
    'Sponsorship 💰💰': 'Funding',
    '🙏 Donation':'Funding',
    '💝 Venue Gifts':'Events',
    'Food / Nutrition Books': 'Misc',
    'UMIF':'Misc',
    '😭 Lost & Found':'Misc',
    '⛺ Outernet':'Teambuilding',
    '😋Snacks':'Food',
    'Registration':'Events',
    'yay':'Misc',
    'Website':'IT',
    'Inner Peace':'Labor',
    'Travel':'Travel',
    '📸 Photography':'Marketing',
    'VEX':'Misc',
    '🖼️Sticker':'Marketing',
    'Subscription':'Subscription',
    'Labor':'Labor',
    'IT':'IT',
    'Events':'Events',
    'Operations':'Operations',
    'Travel':'Travel',
    'Food':'Food',
    'Marketing':'Marketing',
    'Teambuilding':'Teambuilding',
    'Reimbursement':'Reimbursement',
    'Misc':'Misc',
    'Subscription':'Subscription',
    'Tshirts/Hoodies':'Marketing',
    'ComicConBakeMay2023&Tshirt':'Marketing'}

In [160]:
# for transaction in small_data:
#   transaction['tags'][0] = label_map[transaction['tags'][0]]

KeyError: 'Taxes'

## Test Train Split

In [10]:
train,val = train_test_split(small_data, test_size=0.2, random_state=np.random.seed(10), shuffle=True)

In [11]:
val,test = train_test_split(val, test_size=0.5, random_state=np.random.seed(10), shuffle=True)

In [12]:
print(len(train))
print(len(val))
print(len(test))

1422
178
178


## Configuring Data

In [35]:
# function to get unique values
def unique(list1):

    # initialize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    return unique_list


In [73]:
train_memo = [transaction["memo"] for transaction in train]

In [74]:
train_labels = [transaction["tags"][0] for transaction in train]

In [161]:
label_list = np.array(unique(train_labels))

#Initial Model: BERT

In [132]:
#specify model
model_checkpoint = "bert-base-cased"

#specify tokenizer
bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

# Initializing a BERT google-bert/bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the google-bert/bert-base-uncased style configuration
bert_model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [120]:
#tokenize data

train_tokenized_memos = bert_tokenizer(train_memo, max_length = 20, return_tensors = 'pt', padding = 'max_length', truncation = True)
train_tokenized_labels = bert_tokenizer(train_labels, max_length = 4, return_tensors = 'pt', padding = 'max_length', truncation = True)['input_ids']

In [121]:
# Custom dataset
class MemoDataset(Dataset):
    def __init__(self, tokenized_inputs, tokenized_labels):
        self.inputs = tokenized_inputs
        self.labels = tokenized_labels

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item['labels'] = self.labels[idx]
        return item

# Create dataset and dataloader
dataset = MemoDataset(train_tokenized_memos, train_tokenized_labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [139]:
#move to dataloader
batch_size = 4
bert_train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # usually we shuffle, but we shuffled already above
# bert_test_loader = DataLoader(bert_test_data, batch_size=batch_size, shuffle=False)

In [135]:
%%capture

class BERTClassificationNetworkClass(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = bert_model
        self.linear = torch.nn.Linear(768, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        # Get the forward pass. Apply the BERT model, then the linear layer, and
        # then apply the sigmoid
        bert_output = self.model(**x)
        last_hidden_states = bert_output.last_hidden_state
        cls_output = last_hidden_states[:,0,:]
        linear_output = self.linear(cls_output)
        sigmoid_output = self.sigmoid(linear_output)

        return torch.squeeze(sigmoid_output) # removing 'x 1 x ' dimensions


# loss_fn = torch.nn.BCELoss()

bert_classification_model = BERTClassificationNetworkClass().to(device)

In [143]:
test = next(iter(bert_train_loader))

test = {k:v.to(device) for k, v in test.items()}

out = bert_classification_model({'input_ids': test['input_ids'], 'attention_mask': test['attention_mask']})

# loss = loss_fn(out.float(), test['labels'].float())

print('Output: ', out)
# print('Loss: ', loss)


Output:  tensor([0.4799, 0.5355, 0.3709, 0.4383], device='cuda:0',
       grad_fn=<SqueezeBackward0>)


##Next Steps:

* Configure labels to numerical instead of text
* Run BERT model with loss; record for initial untrained performance
* Train
* Incorporate additional data (amount, org_type,...) from NN layer on; potentially add more layers.